# Main Module:

In [10]:
""" 
Main Module as comand line interface
"""
import pandas as pd
import sys
import os.path
import numpy as np

actions = ['analys','test','predict','graph','report']
options = ['pdf','console']
action = None
ds = None
datasetPath='data/StudentsPerformance.csv'

def main() :
    
    print('=====================================')
    for i in range(6) :
        if(i==3) :
            print('|            StudentBAP             |')
            print('|    Student Background Analysis    |')
            print('|    Coppy right Kamel Haoua 2021   |')
            print('|                                   |')
    print('=====================================')
    
    action = getUserAction()
    if(action==None):
        print('Sorry i can\'t understand your action,! see you next time.')
    elif(action=='exit'):
         exit()
    else:
        executeAction(action)
def loadDataset():
    global ds
    if(os.path.exists(datasetPath)):
        ds= pd.read_csv(datasetPath)
        print('Reading dataset ok')
    else:
        print('file '+datasetPath+' doesn\'t exits. Please check file system and try agin later.' )    

#diplay help
def showHelp():
    print('usage :\n'+ \
    'action [options]\n'+ \
    '\n'+ \
    'Where action is one of the following:\n'+ \
    'describe: perfome all statistical analysis and show result as text\n'+ \
    'graph: display graph or show all graphs\n'+ \
    'graphx: display grap  with expanation\n'+ \
    'exit: stop script\n'+ \
    '\n'+ \
    'Option is one of the following:\n'+ \
    'pdf,console, default is console')
    
#return user std inpute text
def getUserAction():
    print('What would like to do now.?')
    maxtry=3;
    while(maxtry>0):
        action = input("action: ")
        print(action)
        if(action==None ):
            print ('No action specified!. type help or shoose one action: \n'+', '.join(actions))
            print()
        elif (len( action)==0) :
            print ('No action specified!. type help or shoose one action: \n'+', '.join(actions))
            print()
        elif(action in actions or action=='exit') :
            return action;
        elif(action=='help') :
            showHelp() 
        else:
            print('Not recognized action. "'+action+'"\n'+'type help or choose one action: \n'+', '.join(actions))
            print()
        if(maxtry==0) :
            return None;
        else:
            maxtry-=1;
                
#execute one action                
def executeAction(act=None):
    if(action=='exit') :
        print('Shutting down kernel. good bye.')
        exit()  
    else :  
        loadDataset()
        cleanUpDataset()
        
        print('executing action: '+act+'\nPlease wait this action may take a while... ')
        if(action=='analys'):
            analys()
        sys.exit()

# verify dataset

"""
 if there is a missing or wrong value values we have to process each column depending on its data type
 if data type is not numeric we remove the row from dataset, if is nemuric we replace
 non consistence values with NaN and replace NaN values with the mean   of that column
"""

def cleanUpDataset(num_columns=None) :
    print('Process missing and or wrong values..')
    print('--------------------------------------------------------')
    columns= ds.columns
    #some column names contain white-space character, so we have to replace it with underscore character(_)
    print('Correcting columns names..')
    ds.columns=ds.columns.str.replace(' ','_')
    ds.columns=ds.columns.str.replace('\/','_')
    print('Replacing  wrong values with null..')
    if(num_columns==None):
        num_columns=ds.select_dtypes(include='int64')
        print('\n')
        print('Numeric columns are')
        print(num_columns.columns)
        for nc in num_columns :
            ds[nc]=ds[nc].apply(lambda x : np.nan if str(type(x))=="<class 'str'>" else x)
    print('\n')
    print('Collecting columns values with null..')    
    columnWithNullValue=list()
    for column in columns :
        datatype=ds[column].dtype
        print('column '+column+'  data type '+str(datatype))
        if (ds[column].isnull().values.any==True):
            columnWithNullValue.append(column)   
    print('---------------------------------------------------------')
    
    if(len(columnWithNullValue)==0):
        print('columns check finish, ok.')
    else :
        print('some columns have null values\n')
        print(columnWithNullValue)   
    print('Replacing numeric null values with mean')
    replcementValues={}
    if(len(columnWithNullValue)>0) :
        for c in columnWithNullValue :
            if(c in num_columns) :
                meanvalue=ds[c].mean(skipna=True)
                replcementValues[c]=meanvalue
    ds.fillna(value=replcementValues, inplace=True)
    print('Ok, replacement done')
    print('Removing rows with null categorical values')
    ds.dropna(inplace=True)
    print('Ok,  Removing done')
    print('\n')

def datasetInfos() :
    if(os.path.exists(datasetPath)) :
        print('File size')
        print(os.stat(datasetPath).st_size)
    if(ds == None) :
        print('No data available yet !.')
        sys.exit()
    else :    
        print('Dataset shape rows (count x comluns count):')
        print(ds.shape)
        print()
        print('Dolumns names:')
        print(ds.columns)
        print()
        print('Numeric columns:')
        print(ds.select_dtypes(include='int64').columns)
        print()
        print('Statistics:')
        print(ds.describe())

#execute the main program
"""
if __name__ == '__main__':
    main()
"""

"\nif __name__ == '__main__':\n    main()\n"

# Dataset balancing check

categrical columns are : ['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch', 'test_preparation_course']

Categorical variables like gender and race/ethnicity should represente the entire population, in other word the sample should not contain signifcant magority or minority, otherwise some alghorithm would yeild not accurate result. Method checkDataBalancing show taht the race/ethnicity variable is not balkanced.


In [13]:
"""
Module analys perfoms statistical Analysis
"""
import sys
"""
check if the data balanced:
In this dataset we have two columns gender and race/ethnicity that are
categorical variable, they should have the same number of values for each category (eg. 500 men and 500 women)
"""
def checkDataBalancing(cat_column=None) :
    if(cat_column==None) :
        print('None categorical column')
        pass
    unique_count = ds[cat_column].nunique()
    ds_size = ds[cat_column].size
    grouped_ds=ds.groupby(by=cat_column)
    normal_count=ds_size/unique_count
    real_count=grouped_ds.size()
    print('values count in each category '+cat_column)
    print(real_count.to_string())
    isBalanced=True
    
    for count in real_count :
        if((count / normal_count) >= 1.1):
            print(count / normal_count)
            isBalanced=False
            break
            
    if(isBalanced==True):
         print('Sample for category '+cat_column+' is balanced')
    else:
         print('Sample for category '+cat_column+' is not balanced,' \
               +'because some '+ cat_column +' contains more than 90% of the total values '+cat_column)
    print()
    
def analys():
    if(ds is None):
        print('Dataset no available')
        sys.exit()
    else:
       
        # find max min values
        print('Student''s gender who get <= 20% math score')
        print(ds.loc[ds['math_score']<=20].reindex(columns=['gender','race_ethnicity'])\
              .groupby(by=['race_ethnicity','gender']).size())
        print('Student''s gender who get >=80% math score')
        print(ds.loc[ds['math_score']>=80].reindex(columns=['gender','race_ethnicity'])\
              .groupby(by=['race_ethnicity','gender']).size())
        print()
        #mean by gender
        print('analysing dataset please wait...')
        print('-----------------------------------------------------')
        print(ds.describe())
        print()    
        print('Mean by gender')
        print(ds.groupby(ds['gender']).mean())
        print()
        print('Mean by race/ethnicity')
        print(ds.groupby(ds['race_ethnicity']).mean())
        print()
        print('Mean by parental level of eductaion')
        print(ds.groupby(ds['parental_level_of_education']).mean())
        print()
        print('Mean by lunch')
        print(ds.groupby(ds['lunch']).mean())
        print()
        print('Mean by test preparation course')
        print(ds.groupby(ds['test_preparation_course']).mean())
        
loadDataset()  
ds.columns=ds.columns.str.replace(' ','_')
ds.columns=ds.columns.str.replace('\/','_')
checkDataBalancing('race_ethnicity')
checkDataBalancing('gender')    
analys()


Reading dataset ok
values count in each category race_ethnicity
race_ethnicity
group A     89
group B    190
group C    319
group D    262
group E    140
1.595
Sample for category race_ethnicity is not balanced,because some race_ethnicity contains more than 90% of the total values race_ethnicity

values count in each category gender
gender
female    518
male      482
Sample for category gender is balanced

Students gender who get <= 20% math score
race_ethnicity  gender
group B         female    3
group C         female    1
dtype: int64
Students gender who get >=80% math score
race_ethnicity  gender
group A         female     3
                male       6
group B         female    11
                male      18
group C         female    22
                male      27
group D         female    17
                male      34
group E         female    21
                male      34
dtype: int64

analysing dataset please wait...
-----------------------------------------------------
 

# **Graphics**

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

#plt.rcParams["figure.figsize"] = [16, 16]
%config InlineBackend.figure_format = 'retina'
sns.set_theme(style="whitegrid")

def showAllGraphs():
    showCategoricalvariables()
    showExtremScores()
    
def showCategoricalvariables():
    # 1) Categorical variables distribution
    #Gender 
    genders= ds.gender.value_counts(ascending=True).to_dict()
    races= ds.race_ethnicity.value_counts(ascending=True).to_dict()
    
    fig, ax=  plt.subplots(nrows=1,ncols=2, figsize=[12,6])
        
    ax[0].pie(x=genders.values(),data=genders, labels=['']* len(genders.keys()),colors=['blue','hotpink'], autopct='%1.1f%%')
    ax[0].axis('equal')
    ax[0].set_title('Gender')
    ax[0].legend(genders.keys())
    
    #Race/Ethnicity
    ax[1].pie(x=races.values(),data=races, labels=['']* len(races.keys()), autopct='%1.1f%%')
    ax[1].axis('equal')
    ax[1].set_title('Races/Ethnicity')
    ax[1].legend(races.keys())
    
    fig.patch.set_facecolor('white')
    fig.suptitle('Categorical variables distribution',fontsize=16)
    fig.subplots_adjust(top=0.88)
    fig.tight_layout()
    plt.show()
    print('------------------------------------------------------------------------')
    
def showExtremScores():
    fig, ax=  plt.subplots(nrows=3,ncols=2, sharey=False, figsize=[16,16])
    fig.suptitle('Extrem scores grouped by races and divided by gender', fontsize=16)
    # math score
    #------------------------------------------------------------------------------------
    math_less_20=ds.loc[ds['math_score']<=20].reindex(columns=['gender','race_ethnicity'])\
              .groupby(by=['race_ethnicity','gender']).size()
    math_df_20=pd.DataFrame({'counts':math_less_20}).reset_index()
    sns.barplot(ax=ax[0,0],palette=['hotpink','blue'], data=math_df_20, x='race_ethnicity', y='counts',hue='gender')
    ax[0,0].set_title('Under 20 math score')
    
    math_greater_80=ds.loc[ds['math_score']>=80].reindex(columns=['gender','race_ethnicity'])\
              .groupby(by=['race_ethnicity','gender']).size()
    math_df_80=pd.DataFrame({'counts':math_greater_80}).reset_index()
    sns.barplot(ax=ax[0,1],palette=['hotpink','blue'], data=math_df_80, x='race_ethnicity', y='counts',hue='gender')
    ax[0,1].set_title('Above 80 math score')
    
    #--------------------------------------------------------------------------------
    # reding score
    
    read_less_20=ds.loc[ds['reading_score']<=20].reindex(columns=['gender','race_ethnicity'])\
              .groupby(by=['race_ethnicity','gender']).size()
    read_df_20=pd.DataFrame({'counts':read_less_20}).reset_index()
    sns.barplot(ax=ax[1,0],palette=['hotpink','blue'], data=read_df_20, x='race_ethnicity', y='counts',hue='gender')
    ax[1,0].set_title('Under 20 reding score')
    
    read_greater_80=ds.loc[ds['reading_score']>=80].reindex(columns=['gender','race_ethnicity'])\
              .groupby(by=['race_ethnicity','gender']).size()
    read_df_80=pd.DataFrame({'counts':read_greater_80}).reset_index()
    sns.barplot(ax=ax[1,1],palette=['hotpink','blue'], data=read_df_80, x='race_ethnicity', y='counts',hue='gender')
    ax[1,1].set_title('Above 80 reding score')
    
    #--------------------------------------------------------------------------------
    # writing score
    
    write_less_20=ds.loc[ds['writing_score']<=20].reindex(columns=['gender','race_ethnicity'])\
              .groupby(by=['race_ethnicity','gender']).size()
    write_df_20=pd.DataFrame({'counts':write_less_20}).reset_index()
    sns.barplot(ax=ax[2,0],palette=['hotpink','blue'], data=write_df_20, x='race_ethnicity', y='counts',hue='gender')
    ax[2,0].set_title('Under 20 writing score')
    
    write_greater_80=ds.loc[ds['writing_score']>=80].reindex(columns=['gender','race_ethnicity'])\
              .groupby(by=['race_ethnicity','gender']).size()
    write_df_80=pd.DataFrame({'counts':write_greater_80}).reset_index()
    sns.barplot(ax=ax[2,1],palette=['hotpink','blue'], data=write_df_80, x='race_ethnicity', y='counts',hue='gender')
    ax[2,1].set_title('Above 80 writing score')
    
    plt.show()

def meanGraph():
    fig, ax = plt.subplots(nrows=3,ncols=2, sharey=False, figsize=[20,16])
    ax[-1, -1].axis('off')
    fig.suptitle('Means grouped by categorical variables', fontsize=16)
        
    by_gender= ds.groupby(ds['gender']).mean().reset_index()
    by_gender=by_gender.melt('gender',var_name='means',value_name='score_means')
    sns.barplot(ax=ax[0,0], data=by_gender, x='gender', y='score_means',hue='means')
    ax[0,0].set_title('mean by gender')
        
    by_race= ds.groupby(ds['race_ethnicity']).mean().reset_index()
    by_race=by_race.melt('race_ethnicity',var_name='means',value_name='score_means')
    sns.barplot(ax=ax[0,1], data=by_race, x='race_ethnicity', y='score_means',hue='means')
    ax[0,1].set_title('mean by race_ethnicity')
        
    by_ple=ds.groupby(ds['parental_level_of_education']).mean().reset_index()
    by_ple=by_ple.melt('parental_level_of_education',var_name='means',value_name='score_means')
    sns.barplot(ax=ax[1,0], data=by_ple, x='parental_level_of_education', y='score_means',hue='means')
    ax[1,0].set_title('mean by parental_level_of_education')    
       
    by_lunch=ds.groupby(ds['lunch']).mean().reset_index()
    by_lunch=by_lunch.melt('lunch',var_name='means',value_name='score_means')
    sns.barplot(ax=ax[1,1], data=by_lunch, x='lunch', y='score_means',hue='means')
    ax[1,1].set_title('Mean by lunch type')    
       
    by_tpc=ds.groupby(ds['test_preparation_course']).mean().reset_index()
    by_tpc=by_tpc.melt('test_preparation_course',var_name='means',value_name='score_means')
    sns.barplot(ax=ax[2,0], data=by_tpc, x='test_preparation_course', y='score_means',hue='means')
    ax[2,0].set_title('Mean by test_preparation_course') 
    
    plt.show()
    
def boxGraphs():
    fig, ax = plt.subplots(nrows=5, ncols=3, figsize=[30,20])
    fig.suptitle('Grouped by scores', fontsize=16)
        
    sns.boxplot(ax=ax[0,0],y='gender',x='math_score', data=ds, palette=['hotpink','blue'])
    sns.boxplot(ax=ax[0,1],y='gender',x='reading_score', data=ds, palette=['hotpink','blue'])
    sns.boxplot(ax=ax[0,2],y='gender',x='writing_score', data=ds, palette=['hotpink','blue'])
    
    sns.boxplot(ax=ax[1,0],y='race_ethnicity',x='math_score', data=ds)
    sns.boxplot(ax=ax[1,1],y='race_ethnicity',x='reading_score', data=ds)
    sns.boxplot(ax=ax[1,2],y='race_ethnicity',x='writing_score', data=ds)
    
    sns.boxplot(ax=ax[2,0],y='parental_level_of_education',x='math_score', data=ds)
    sns.boxplot(ax=ax[2,1],y='parental_level_of_education',x='reading_score', data=ds)
    sns.boxplot(ax=ax[2,2],y='parental_level_of_education',x='writing_score', data=ds)
    
    sns.boxplot(ax=ax[3,0],y='lunch',x='math_score', data=ds)
    sns.boxplot(ax=ax[3,1],y='lunch',x='reading_score', data=ds)
    sns.boxplot(ax=ax[3,2],y='lunch',x='writing_score', data=ds)
    
    sns.boxplot(ax=ax[4,0],y='test_preparation_course',x='math_score', data=ds)
    sns.boxplot(ax=ax[4,1],y='test_preparation_course',x='reading_score', data=ds)
    sns.boxplot(ax=ax[4,2],y='test_preparation_course',x='writing_score', data=ds)
    
    plt.show() 
    
def barMeanPerformanceGraph():
    fig, ax = plt.subplots(nrows=5, ncols=4, figsize=[40,40])
    plt.xticks(rotation=45)
    fig.suptitle('Mean and perfomances of three exams', fontsize=28)
    
    # Mean
    mean_perormance=ds.eval('(math_score + reading_score + writing_score)/3').rename('mean_perormance')
    sns.barplot(ax=ax[0,0], data=ds,y=mean_perormance , x='gender',palette=['hotpink','blue']).set_title('Mean for all scores', fontsize=18)
    sns.barplot(ax=ax[1,0], data=ds,y=mean_perormance , x='race_ethnicity')
    ple=sns.barplot(ax=ax[2,0], data=ds,y=mean_perormance , x='parental_level_of_education')
    ple.set_xticklabels(ple.get_xticklabels(),rotation=30)
    
    sns.barplot(ax=ax[3,0], data=ds,y=mean_perormance , x='lunch')
    sns.barplot(ax=ax[4,0], data=ds,y=mean_perormance , x='test_preparation_course')
   
    # Math score
    sns.barplot(ax=ax[0,1], data=ds,y='math_score' , x='gender',palette=['hotpink','blue']).set_title('Math score', fontsize=18)
    sns.barplot(ax=ax[1,1], data=ds,y='math_score' , x='race_ethnicity')
    sns.barplot(ax=ax[2,1], data=ds,y='math_score' , x='parental_level_of_education').set_xticklabels(ple.get_xticklabels(),rotation=30)
    sns.barplot(ax=ax[3,1], data=ds,y='math_score' , x='lunch')
    sns.barplot(ax=ax[4,1], data=ds,y='math_score' , x='test_preparation_course')
    
    # Reading score
    sns.barplot(ax=ax[0,2], data=ds,y='reading_score' , x='gender',palette=['hotpink','blue']).set_title('Reading score', fontsize=18)
    sns.barplot(ax=ax[1,2], data=ds,y='reading_score' , x='race_ethnicity')
    sns.barplot(ax=ax[2,2], data=ds,y='reading_score' , x='parental_level_of_education').set_xticklabels(ple.get_xticklabels(),rotation=30)
    sns.barplot(ax=ax[3,2], data=ds,y='reading_score' , x='lunch')
    sns.barplot(ax=ax[4,2], data=ds,y='reading_score' , x='test_preparation_course')
    
    # Writing scores
    sns.barplot(ax=ax[0,3], data=ds,y='writing_score' , x='gender',palette=['hotpink','blue']).set_title('Writing scores', fontsize=18)
    sns.barplot(ax=ax[1,3], data=ds,y='writing_score' , x='race_ethnicity')
    sns.barplot(ax=ax[2,3], data=ds,y='writing_score' , x='parental_level_of_education').set_xticklabels(ple.get_xticklabels(),rotation=30)
    sns.barplot(ax=ax[3,3], data=ds,y='writing_score' , x='lunch')
    sns.barplot(ax=ax[4,3], data=ds,y='writing_score' , x='test_preparation_course')
    
barMeanPerformanceGraph()    


# Machine Learning

In this module we gonna be using RandomForestClassifier to predict scores.


*   Transform categorical variable into numeric values using OneHotEncoder
*   Devide  dataset into two subdatasets (80/20 %) one for training model and other for testing.







In [113]:
"""
Machine learning p
"""
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Load pandas
import pandas as pd
# Load numpy
import numpy as np
# Set random seed, so it start from 0
np.random.seed(0)
# 1) math_score machine learning
math_target = ds.math_score
#split dataset 
x_train, x_test, y_train, y_test= train_test_split(ds,math_target,test_size=0.2, random_state =0)
# preparing features
gender=x_train['gender'].unique()
race_ethnicity=x_train['race_ethnicity'].unique()
parental_level_of_education=x_train['parental_level_of_education'].unique()
lunch=x_train['lunch'].unique()
test_preparation_course=x_train['test_preparation_course'].unique()
#transforming features values from string to numeric
ohe=OneHotEncoder(categories=[gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course])
cat_ds = x_train[['gender','race_ethnicity','parental_level_of_education','lunch','test_preparation_course']]
ohe.fit(cat_ds)
OneHotEncoder()
features = ohe.transform(cat_ds).toarray()
trfm=ohe.transform([['male','group E','some high school','standard','none']]).toarray()
clf=RandomForestClassifier(n_jobs=2, random_state=0)

clf.fit(np.asarray(features),y_train)
clf.predict(trfm)



array([82])

In [ ]:
def printReport():
    option=input('please make a choice: pdf or console:')
    if(option=='pdf'):
        pass
    else :    
        pass

# **Test statistics:**

in this module we try to find out if there is a correlation between a given category of student and any of the performances, so we have to filter out a category and compare them against each another.

In [ ]:
"""
This module run tests for each categorical variable
"""
from scipy.stats import mannwhitneyu
import pandas as pd

def runtest(dist1,dist2):
    alpha=0.05
    stat,pv= mannwhitneyu(dist1,dist2,alternative='two-sided')
    if(pv<alpha):
        return True
    else:
        return False
    
def runAllTests():
    principal_columns=['math_score','reading_score','writing_score']
    males=ds[ds['gender']=='male'][principal_columns]
    females=ds[ds['gender']=='female'][principal_columns]   
    
    print('---------Gender statitcs test:----------')
    math_test=runtest(males['math_score'],females['math_score'])
    writing_test=runtest(males['reading_score'],females['reading_score'])
    redaing_test=runtest(males['writing_score'],females['writing_score'])
        
    print('Male are '+('good than 'if(math_test)else 'equal with'  )+'females in math ')
    print('Females are '+('good than 'if(writing_test)else 'equal with '  )+'males in writing ')
    print('Females are '+('good than 'if(redaing_test)else 'equal with '  )+'males in reading ')
    print()
    print('---------race_ethnicity statitcs test---------')
    print()
    group_a=ds[ds['race_ethnicity']=='group A'][principal_columns]
    group_b=ds[ds['race_ethnicity']=='group B'][principal_columns]
    group_c=ds[ds['race_ethnicity']=='group C'][principal_columns]
    group_d=ds[ds['race_ethnicity']=='group D'][principal_columns]
    group_e=ds[ds['race_ethnicity']=='group E'][principal_columns]
    
    math_test=runtest(group_e['math_score'],group_d['math_score'])
    writing_test=runtest(group_e['reading_score'],group_d['reading_score'])
    redaing_test=runtest(group_e['writing_score'],group_d['writing_score'])
    
    print('Group E vs group D:')
    
    print('group E is '+('good than 'if(math_test)else 'similar with'  )+'group D in math ')
    print('group E is '+('good than 'if(writing_test)else 'equal with '  )+'group D in writing ')
    print('group E is '+('good than 'if(redaing_test)else 'equal with '  )+'group D in reading ') 
    print()
    print('Group D vs group C:')
    math_test=runtest(group_d['math_score'],group_c['math_score'])
    writing_test=runtest(group_d['reading_score'],group_c['reading_score'])
    redaing_test=runtest(group_d['writing_score'],group_c['writing_score'])
    
    print('group D is '+('good than 'if(math_test)else 'similar with'  )+'group C in math ')
    print('group D is '+('good than 'if(writing_test)else 'similar with '  )+'group C in writing ')
    print('group D is '+('good than 'if(redaing_test)else 'similar with '  )+'group C in reading ')
    print()
    print('---------Parental level of education---------')
    print()
    bachelor_degree=ds[ds['parental_level_of_education']=='bachelor''s degree'][principal_columns]
    master_degree=ds[ds['parental_level_of_education']=='master''s degree'][principal_columns]
    ass_degree=ds[ds['parental_level_of_education']=='associate''s degree'][principal_columns]
    somme_college=ds[ds['parental_level_of_education']=='some college'][principal_columns]
    high_school=ds[ds['parental_level_of_education']=='high school'][principal_columns]
    somme_high_school=ds[ds['parental_level_of_education']=='some high school'][principal_columns]
    
    print('bachelor degree vs master degree:')
    math_test=runtest(bachelor_degree['math_score'],master_degree['math_score'])
    writing_test=runtest(bachelor_degree['reading_score'],master_degree['reading_score'])
    redaing_test=runtest(bachelor_degree['writing_score'],master_degree['writing_score'])
    print('master degree is '+('good than 'if(math_test)else 'similar with'  )+'bachelor degree in math ')
    print('master degree is '+('good than 'if(writing_test)else 'similar with '  )+'bachelor degree in writing ')
    print('master degree is '+('good than 'if(redaing_test)else 'similar with '  )+'bachelor degree in reading ')
    print()
    
    print('associate degree vs some college:')
    math_test=runtest(ass_degree['math_score'],somme_college['math_score'])
    writing_test=runtest(ass_degree['reading_score'],somme_college['reading_score'])
    redaing_test=runtest(ass_degree['writing_score'],somme_college['writing_score'])
    print('associate degree is '+('good than 'if(math_test)else 'similar with'  )+'some college in math ')
    print('associate degree is '+('good than 'if(writing_test)else 'similar with '  )+'some college in writing ')
    print('associate degree is '+('good than 'if(redaing_test)else 'similar with '  )+'some college in reading ')
    print()
    
    print('some high school vs high school:')
    math_test=runtest(high_school['math_score'],somme_high_school['math_score'])
    writing_test=runtest(high_school['reading_score'],somme_high_school['reading_score'])
    redaing_test=runtest(high_school['writing_score'],somme_high_school['writing_score'])
    print('some high school is '+('good than 'if(math_test)else 'similar with '  )+'high school in math ')
    print('some high school is '+('good than 'if(writing_test)else 'similar with '  )+'high school in writing ')
    print('some high school is '+('good than 'if(redaing_test)else 'similar with '  )+'high school in reading ')
    print()
    
runAllTests()        